In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
import pandas as pd
from keras.models import load_model
BATCH_SIZE = 16
NUM_CLASSES = 10
NUM_EPOCHS = 10


Using TensorFlow backend.


In [2]:
class Game:

    def __init__(self, size=4, score_to_win=None, rate_2=0.5, random=False, enable_rewrite_board=False):
        '''

        :param size: the size of the board
        :param score_to_win: the terminate score to indicate `win`
        :param rate_2: the probability of the next element to be 2 (otherwise 4)
        :param random: a random initialized board (a harder mode)
        '''
        self.size = size
        if score_to_win is None:
            score_to_win = np.inf
        self.score_to_win = score_to_win
        self.__rate_2 = rate_2
        if random:
            self.__board = \
                2 ** np.random.randint(1, 10, size=(self.size, self.size))
            self.__end = False
        else:
            self.__board = np.zeros((self.size, self.size))
            # initilize the board (with 2 entries)
            self._maybe_new_entry()
            self._maybe_new_entry()
        self.enable_rewrite_board = enable_rewrite_board
        assert not self.end

    def move(self, direction):
        '''
        direction:
            0: left
            1: down
            2: right
            3: up
        '''
        # treat all direction as left (by rotation)
        board_to_left = np.rot90(self.board, -direction)
        for row in range(self.size):
            core = _merge(board_to_left[row])
            board_to_left[row, :len(core)] = core
            board_to_left[row, len(core):] = 0

        # rotation to the original
        self.__board = np.rot90(board_to_left, direction)
        self._maybe_new_entry()

    def __str__(self):
        board = "State:"
        for row in self.board:
            board += ('\t' + '{:8d}' *
                      self.size + '\n').format(*map(int, row))
        board += "Score: {0:d}".format(self.score)
        return board

    @property
    def board(self):
        '''`NOTE`: Setting board by indexing,
        i.e. board[1,3]=2, will not raise error.'''
        return self.__board.copy()

    @board.setter
    def board(self, x):
        if self.enable_rewrite_board:
            assert self.__board.shape == x.shape
            self.__board = x.astype(self.__board.dtype)
        else:
            print("Disable to rewrite `board` manually.")

    @property
    def score(self):
        return int(self.board.max())

    @property
    def end(self):
        '''
        0: continue
        1: lose
        2: win
        '''
        if self.score >= self.score_to_win:
            return 2
        elif self.__end:
            return 1
        else:
            return 0

    def _maybe_new_entry(self):
        '''maybe set a new entry 2 / 4 according to `rate_2`'''
        where_empty = self._where_empty()
        if where_empty:
            selected = where_empty[np.random.randint(0, len(where_empty))]
            self.__board[selected] = \
                2 if np.random.random() < self.__rate_2 else 4
            self.__end = False
        else:
            self.__end = True

    def _where_empty(self):
        '''return where is empty in the board'''
        return list(zip(*np.where(self.board == 0)))


def _merge(row):
    '''merge the row, there may be some improvement'''
    non_zero = row[row != 0]  # remove zeros
    core = [None]
    for elem in non_zero:
        if core[-1] is None:
            core[-1] = elem
        elif core[-1] == elem:
            core[-1] = 2 * elem
            core.append(None)
        else:
            core.append(elem)
    if core[-1] is None:
        core.pop()
    return core


In [3]:
out_shape = (4,4)
CAND = 16
map_table = {2**i:i for i in range(1,CAND)}
map_table[0]=0

def NB_board(arr):
    ret = np.zeros(shape=out_shape+(CAND,),dtype=bool)
    for r in range (out_shape[0]):
        for c in range (out_shape[1]):
            ret[r,c,map_table[arr[r,c]]]=1
    ret=ret.reshape(1,4,4,16)
    return ret

In [4]:
from IPython.display import HTML, display as ipy_display
import pandas as pd
class Display:
    '''A basic display.'''

    def display(self, game):
        if game.end == 2:
            self.win(game)
        elif game.end == 1:
            self.lose(game)
        else:
            self.show(game)


    def _display(self, game):
        print(game)
       # print(game.board)
       # dataframe = pd.DataFrame({'board':game.board.reshape(1,)},index=[0])
       # dataframe.to_csv("train.csv",index = False,sep=',')
    def show(self, game):
        self._display(game)

    def win(self, game):
        self._display(game)
        print("You win! Score: %s" % game.score)

    def lose(self, game):
        self._display(game)
        print("You lose! Score: %s" % game.score)


class IPythonDisplay(Display):
    '''A better display for IPython (Jupyter) notebook environments.'''

    def __init__(self, display_size=40):
        self.display_size = display_size

    def _render(self, game):
        board = game.board
        html = '''<h1>Score: {}</h1>'''.format(game.score)
        table = '''<table style="border: 5px solid black;">{}</table>'''
        td = '''<td style="border:3px solid black; text-align:center;"
         width="%s" height="%s">{}</td>''' % (self.display_size, self.display_size)
        content = ''
        for row in range(game.size):
            content += '''<tr>'''
            for col in range(game.size):
                elem = int(board[row, col])
                content += td.format(elem if elem else "")
            content += '''</tr>'''
        html += table.format(content)
        return html

    def _display(self, game):
        if 'ipykernel' in sys.modules:
            source = self._render(game)
            ipy_display(HTML(source))
        else:
            print("Warning: since it's not in ipykernel, "
                  "it will show the command line version.")
            super()._display(game)


In [5]:

import csv
class Agent:
    '''Agent Base.'''
    def __init__(self, game, display=None):
        self.game = game
        self.display = display

    def play(self, max_iter=np.inf, verbose=False):
        n_iter = 0
        while (n_iter < max_iter) and (not self.game.end):
            direction = self.step()
            self.game.move(direction)
            n_iter += 1
            if verbose:
                #print("Iter: {}".format(n_iter))
                #print("======Direction: {}======".format(
                    #["left", "down", "right", "up"][direction]))
                if self.display is not None:
                    self.display.display(self.game)
        #print("Iter: {}".format(n_iter))
        
    def step(self):
        direction = int(input("0: left, 1: down, 2: right, 3: up = ")) % 4
        return direction

In [6]:
class the_model_32:
    def __init__(self):
        self.model=model_32
    def predict(self,board):
        Real_board = NB_board(board)
        direction = self.model.predict(Real_board)
        move = direction.argmax()
        return move    

In [7]:
class Test_Agent(Agent):

    def __init__(self, game, display=None):
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)
        self.search_func_32 = the_model_32()
          
    def step(self):                  
        direction = self.search_func_32.predict(self.game.board) 
        return direction

In [12]:
for j in range (0,1):
    time=50
    model_32 = load_model('final.h5')
    times_32 = 0
    times_64 = 0
    times_128 = 0
    times_256 = 0
    times_512 = 0
    times_1024 =0
    times_2048 = 0
    score = 0
    for i in range(0,time):
        game = Game(4, random=False)
        agent = Test_Agent(game)
        agent.play(verbose=True)
        print (game.score)
        if game.score >=32:
            times_32 = times_32 +1 
        if game.score >=64:
            times_64 = times_64 + 1
        if game.score >=128:
            times_128 = times_128+1
        if game.score >=256:
            times_256 = times_256 + 1
        if game.score >=512:
            times_512 = times_512 +1
        if game.score >= 1024:
            times_1024 = times_1024+1
        if game.score >= 2048:
            times_2048 = times_2048+1
        score = score + game.score
        i=i+1
        print("-----------------------第",i,"次迭代---------------------------")
    print("到达32分次数为：",times_32)
    print("到达64分次数为：",times_64)
    print("到达128分次数为：",times_128)
    print("到达256分次数为：",times_256)
    print("到达512分次数为：",times_512)
    print("到达1024分次数为：",times_1024)
    print("到达2048分次数为：",times_2048)
    print (score/time)

256
-----------------------第 1 次迭代---------------------------
1024
-----------------------第 2 次迭代---------------------------
1024
-----------------------第 3 次迭代---------------------------
128
-----------------------第 4 次迭代---------------------------
1024
-----------------------第 5 次迭代---------------------------
1024
-----------------------第 6 次迭代---------------------------
1024
-----------------------第 7 次迭代---------------------------
512
-----------------------第 8 次迭代---------------------------
1024
-----------------------第 9 次迭代---------------------------
1024
-----------------------第 10 次迭代---------------------------
2048
-----------------------第 11 次迭代---------------------------
1024
-----------------------第 12 次迭代---------------------------
512
-----------------------第 13 次迭代---------------------------
1024
-----------------------第 14 次迭代---------------------------
1024
-----------------------第 15 次迭代---------------------------
256
-----------------------第 16 次迭代------------------